# Lab Assigment 2 Estimation of Apparent Motion

<p style="text-align:left;">
    José Pedro Cruz
    <span style="float:right;">
        up201504646
    </span>
</p>
<p style="text-align:left;">
    Martinho Figueiredo
    <span style="float:right;">
        up201506179
    </span>
</p>
<p style="text-align:left;">
    Nuno Nascimento
    <span style="float:right;">
        up201907933
    </span>
</p>


[![GitHub](https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white)](https://github.com/martinhofigueiredo/VC)



## Scheme
```mermaid
flowchart LR
    A[Ingest Footage] -->|mp4 or mpegs| B{Multi Channel?}
    B -->|Yes| C[Split into Channels]
    B -->|No| D[Gray]-->H
    C --> E[R]-->H
    C --> F[G]-->H
    C --> G[B]-->H
    H{Multi Resolution?}-->|Yes|I[n Pyramid DownSampling Average]-->K
    H-->|No|J[One Shot]-->K
    K{algo} -->|HornShunck| M((FLOW))
    K -->|LucasKanade| M
```

In [2]:
import cv2
import numpy as np
import yaml
import os
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import struct
import argparse
%matplotlib widget


In [3]:
# Loads a yaml file with the parameters to run the code
def load_config(filename):
    with open(filename, 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    return config

In [4]:
# Find all jpgs and pngs in a folder and returns a list of their respective path 
def get_frame_paths(folder_path):
    frame_paths = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            frame_path = os.path.join(folder_path, filename)
            frame_paths.append(frame_path)
    frame_paths.sort()
    return frame_paths

In [5]:
# Checks if the inut file is a directory and if it is it will try and get the frames inside the folder
# it will check if it is and mp4 and if it is it will read it and create a list of frames
def get_input_frames(input_path):
    frames = []
    if os.path.isdir(input_path):
        frame_paths = get_frame_paths(input_path)
        print(f"{frame_paths}")
        frames = [cv2.imread(frame_path) for frame_path in frame_paths]
    else :
        if input_path.endswith('.mp4'):
            cap = cv2.VideoCapture(input_path)
            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            cap.release()
    return frames

In [6]:
def write_flow(filename, flow):
    height, width = flow.shape[:2]
    with open(filename, 'wb') as f:
        f.write(struct.pack('f', width))
        f.write(struct.pack('f', height))
        f.write(flow.astype(np.float32).tobytes())
  

In [8]:
def read_flow(filename):
    with open(filename, 'rb') as f:
        width = struct.unpack('f', f.read(4))[0]
        height = struct.unpack('f', f.read(4))[0]
        flow_bytes = f.read()

    flow = np.frombuffer(flow_bytes, np.float32)
    flow = flow.reshape(int(height), int(width), 2)

    return flow

def compute_angular_error(flow_gt, flow_est):
    angles_gt = np.arctan2(flow_gt[:, :, 1], flow_gt[:, :, 0])
    angles_est = np.arctan2(flow_est[:, :, 1], flow_est[:, :, 0])
    error = (np.sum((angles_gt - angles_est) ** 2, axis=2))
    return np.mean(error)

def compute_endpoint_error(flow_gt, flow_est):
    error = np.sqrt(np.sum((flow_gt - flow_est) ** 2, axis=2))
    return np.mean(error)

def compare_flow_files(file_gt, file_est):
    flow_gt = read_flow(file_gt)
    flow_est = read_flow(file_est)

    error_endpoint = compute_endpoint_error(flow_gt, flow_est)
    error_angular = compute_angular_error(flow_gt, flow_est)

    return error_endpoint, error_angular

# Lucas-Kanade

In [6]:
import struct

def save_flow_field(flow, filename):
    height, width, _ = flow.shape
    with open(filename, 'wb') as f:
        # Write the .flo file header
        f.write(struct.pack('f', 1212500304.0))  # Magic number
        f.write(struct.pack('i', width))  # Image width
        f.write(struct.pack('i', height))  # Image height

        # Write the flow vectors
        f.write(flow.astype(np.float32).tobytes())


In [7]:
print(f"Loading File \'config_LK.yml\'")

config = load_config('config_LK.yml')

input_path = os.getcwd()+config['input_path']

frames = get_input_frames(input_path)


feature_params = dict(
    maxCorners=config['max_corners'],
    qualityLevel=config['quality_level'],
    minDistance=config['min_distance'],
    blockSize=config['block_size']
)
lk_params = dict(
    winSize=(config['window_size'], config['window_size']),
    maxLevel=config['max_level'],
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, config['max_iterations'], config['epsilon'])
)
#############
color = np.random.randint(0, 255, (100, 3))
old_frame = frames[0]
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

mask = np.zeros_like(old_frame)

for frame_num in frames[1:]:
    
    current_frame = frame_num
    frame_gray = cv2.cvtColor(frame_num, cv2.COLOR_BGR2GRAY)
    
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        
    frame_with_flow = cv2.add(current_frame, mask)
    #cv2.imshow('frame', frame_with_flow)
    #cv2.waitKey(0)
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
#cv2.imwrite('flow.png', frame_with_flow)
save_flow_field(mask, 'flow_field.flo')
#############

Loading File 'config_LK.yml'
['c:\\Users\\nmcna\\Desktop\\FEUP\\4oano\\2osemestre\\VC\\assignment2\\dataset\\other-data\\Beanbags\\frame07.png', 'c:\\Users\\nmcna\\Desktop\\FEUP\\4oano\\2osemestre\\VC\\assignment2\\dataset\\other-data\\Beanbags\\frame08.png', 'c:\\Users\\nmcna\\Desktop\\FEUP\\4oano\\2osemestre\\VC\\assignment2\\dataset\\other-data\\Beanbags\\frame09.png', 'c:\\Users\\nmcna\\Desktop\\FEUP\\4oano\\2osemestre\\VC\\assignment2\\dataset\\other-data\\Beanbags\\frame10.png', 'c:\\Users\\nmcna\\Desktop\\FEUP\\4oano\\2osemestre\\VC\\assignment2\\dataset\\other-data\\Beanbags\\frame11.png', 'c:\\Users\\nmcna\\Desktop\\FEUP\\4oano\\2osemestre\\VC\\assignment2\\dataset\\other-data\\Beanbags\\frame12.png', 'c:\\Users\\nmcna\\Desktop\\FEUP\\4oano\\2osemestre\\VC\\assignment2\\dataset\\other-data\\Beanbags\\frame13.png', 'c:\\Users\\nmcna\\Desktop\\FEUP\\4oano\\2osemestre\\VC\\assignment2\\dataset\\other-data\\Beanbags\\frame14.png']


In [8]:
# Function to read the optical flow field from a .flo file
def read_flow_field(filename):
    with open(filename, 'rb') as f:
        magic = np.frombuffer(f.read(4), dtype=np.uint8).view(np.int32)
        print(magic)
        if magic != 1318095507:
            raise ValueError("Invalid .flo file format!")

        width = np.frombuffer(f.read(4), dtype=np.int32)[0]
        height = np.frombuffer(f.read(4), dtype=np.int32)[0]
        print(width)
        print(height)
        flow_data = np.frombuffer(f.read(), dtype=np.float32).reshape(width,height,3)

        #non_zero_values = flow_data[np.nonzero(flow_data)]  # Extract non-zero values
        
        #print(non_zero_values)
        print(np.asarray(flow_data).shape)
    return flow_data

# Example usage
flow_data = read_flow_field('flow_field.flo')  # Read the .flo file

# Compute the magnitude and angle of the flow vectors
magnitude, angle = cv2.cartToPolar(flow_data[..., 0], flow_data[..., 1])

# Visualize the flow field
hue = angle * 180 / np.pi / 2
saturation = np.ones_like(magnitude)
value = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
flow_visualization = cv2.cvtColor(cv2.merge((hue, saturation, value)).astype(np.uint8), cv2.COLOR_HSV2BGR)

# Display the flow visualization
cv2.imshow('Optical Flow', flow_visualization)
cv2.waitKey(0)
cv2.destroyAllWindows()


[1318095507]
640
480
(640, 480, 3)


In [ ]:
prev_frame = frames[0]
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
prev_roi = cv2.boundingRect(prev_corners)

flow_list = []  #List to store optical flow vectors

mask = np.zeros_like(prev_frame)

fig, ax = plt.subplots(figsize=(10, 7))

for curr_frame in frames[1:]:
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    curr_corners, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_corners, None, **lk_params)

    good_prev_corners = prev_corners[status == 1]
    good_curr_corners = curr_corners[status == 1]

    flow = good_curr_corners - good_prev_corners # Calculate optical flow vectores
    
    flow_list.append(flow)

    M, _ = cv2.findHomography(good_prev_corners, good_curr_corners, cv2.RANSAC, config['ransac_threshold'])

    warped_roi = cv2.warpPerspective(prev_frame, M, (curr_frame.shape[1], curr_frame.shape[0]))
    warped_mask = cv2.warpPerspective(mask, M, (curr_frame.shape[1], curr_frame.shape[0]))

    output = curr_frame.copy()
    output[warped_mask != 0] = 0
    output = cv2.add(output, warped_roi)

    ax.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
    ax.axis('off')
    plt.tight_layout()
    clear_output(wait=True)
    display(fig)

    prev_gray = curr_gray.copy()
    prev_corners = good_curr_corners.reshape(-1, 1, 2)
    prev_roi = cv2.boundingRect(prev_corners)
flow_array = np.array(flow_list)

output_file = 'Optical_flow.flo'
write_flow(output_file,flow_array)
cv2.destroyAllWindows()

# Horn-Shunchk

# Bench Marking

## Import Dataset

In [74]:
# IMport dataset with public ground truth

import os
import subprocess
import wget
import zipfile

# Specify the URLs of the files to download
url1 = "https://vision.middlebury.edu/flow/data/comp/zip/other-gt-flow.zip"
url2 = "https://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip"
url3 = "https://vision.middlebury.edu/flow/data/comp/zip/eval-color-allframes.zip"
# Specify the destination folder to store the downloaded files
destination_folder = "dataset/"

# Download the files using wget
file1 = wget.download(url1, out=destination_folder)
file2 = wget.download(url2, out=destination_folder)
file3 = wget.download(url3, out=destination_folder)

# Unzip the files
with zipfile.ZipFile(file1, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

with zipfile.ZipFile(file2, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)
    
with zipfile.ZipFile(file3, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

# Remove the zip files if needed
os.remove(file1)
os.remove(file2)
os.remove(file3)